# 0. Setup

### Import Required Libaried


In [1]:
import os
import openai

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

import dotenv 
#load the environment variables of .env file
%load_ext dotenv
%dotenv

### Setup Azure Credentials


In [2]:
# Setup the required credential for using Azure cognitive search
search_endpoint = f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net/"
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
search_client = SearchClient(endpoint=search_endpoint,
                                index_name=os.getenv("AZURE_SEARCH_INDEX"),
                                credential=search_creds)

# Setup the required credential for using Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")     
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-05-15"

### Setup Embedding model


In [3]:
def compute_embedding(text):
    return openai.Embedding.create(engine="embedding", input=text)["data"][0]["embedding"]

def nonewlines(s: str) -> str:
    return s.replace(' ', ' ').replace('\r', ' ')

# 1. Using ChatGPT Through calling API

In [9]:
#change the query to what you want to ask chatGPT
query = "Who is Ed Sheeran??"

messages = [
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
chat_content

'Ed Sheeran is an English singer, songwriter, musician, and actor. He was born on February 17, 1991, in Halifax, West Yorkshire, England. Sheeran gained popularity in 2011 with his debut album "+", which included hit singles like "The A Team" and "Lego House". He has since released several successful albums, including "X" (2014) and "÷" (2017), which featured chart-topping songs like "Shape of You" and "Perfect". Sheeran is known for his acoustic guitar-driven pop songs and his soulful voice. He has won numerous awards, including multiple Grammy Awards, and has achieved global success as one of the best-selling music artists in the world.'

### Modifying the role of ChatGPT 
- Adding system message
- Adding few shot example

In [5]:
#change the query to what you want to ask chatGPT
query = "Write a poem about university life"

#change the systemMessage to how you want chatGPT to behave
systemMessage = '''You are a Shakespearean writing assistant who speaks in a Shakespearean style. 
                    You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
                    Here are some example of Shakespeare's style:
                    - Romeo, Romeo! Wherefore art thou Romeo?
                    - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
                    - Shall I compare thee to a summer's day? Thou art more lovely and more temperate.'''

messages = [
    {'role' : 'system', 'content' : systemMessage},
    #change the content here to your example question
    {'role' : 'user', 'content' : 'Please write a short text turning down an invitation to dinner.'},
    #change the content here to your example answer
    {'role' : 'assistant', 'content' : '''Dearest,
                                        Regretfully, I must decline thy invitation.
                                        Prior engagements call me hence. Apologies.'''},
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

In fair academe, where knowledge doth reside,
Where students gather, seeking wisdom's tide,
There lies a realm of study and delight,
Where intellect doth shine, a radiant light.

The halls resound with scholarly discourse,
As minds engage, inquisitive, of course.
The lectures, like a symphony of thought,
Inspire minds to grow, to learn, be taught.

From dusty tomes, the ancient wisdom gleams,
As students delve into their chosen themes.
Philosophy, with Socrates' wise voice,
Invites us all to question and rejoice.

In laboratories, science doth unfold,
Experiments and hypotheses untold.
With flask and microscope, we seek to find,
The secrets of the world, both grand and kind.

Yet university life, it hath its charms,
Beyond the classroom, in its open arms.
The friendships forged, a bond that shall endure,
Through laughter, tears, and memories pure.

The campus, with its vibrant tapestry,
Of clubs and sports, a joyful jubilee.
From theatre stages to the football field,
A wealth of talent

# 2. Implementing a RAG model

### Obtain related information using vector search

In [15]:
#change the query to what you want to ask the RAG model
query = "Generate a study plan for studying Python from begineer to advanced."
query_vector = compute_embedding(query)

r = search_client.search(query, 
                        top=3, 
                        vector=query_vector, 
                        top_k=50, 
                        vector_fields="embedding")

results = [doc["sourcepage"] + ": " + nonewlines(doc["content"]) for doc in r]

for result in results:
    print(result)

The_Innovation_Wings-3.pdf:   In this group, we plan to pursue several interdisciplinary projects that will improve the ways we can study and teach about the human past. Our projects will include uses of augmented and virtual reality in recording, teaching, and presenting archaeological sites. We are also studying the use of machine learning and computer vision for several purposes to study visual datasets such as satellite images of ancient landscapes. We work with a field project that travels to Armenia each summer to excavate. We welcome anyone who would like to join our team, from any Faculty. We especially invite engineers and computer scientists, but we also have many other projects which anyone with an interest can join. Let’s learn about the human past together! Thank you! SIG – HKU Astar (formerly RoboMaster ICRA AI Challenge)  HKU Astar is a student group who participates in Robomaster University AI Challenge Competition. Robomaster University AI Challenge is an AI-based robo

### Query the GPT model with query + information

In [21]:
#change the systemMessage to how you want chatGPT to behave
                    #Answer ONLY with the facts listed in the list of sources below. 
systemMessage = """
                Assistant that ONLY answers JSON formatted response. You are a profession teacher that creates designated study plan for students with different needs.
                Return a study plan in JSON format. Below is a situation and an example.
                A student is studying Calculus 1. 
                Below is a general study plan for him in JSON format.
                {
                    "subject": 
                    "plan": {
                        "chapters": {
                            "1": {
                                    "name": "Limit and Continuity",
                                    "time": "60",
                                    "links": ["www.youtube.com", "www.facebook.com"]
                                },
                            "2": {
                                    "name": "Differentiation",
                                    "time": "60",
                                    "links": ["www.youtube.com", "www.facebook.com"]
                                },
                            }
                        }
                    },
                    }
                }
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(results)}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content

import json
def string_to_json_file(string, file_name):
    string = json.loads(string)
    with open(file_name, 'w') as json_file:
        json.dump(string, json_file, indent=4)
# new_content = chat_content.replace('\\', '')
# new_content = new_content.replace('\n', '')
string_to_json_file(chat_content, "output.json")
print(chat_content)

{
    "subject": "Python",
    "plan": {
        "chapters": {
            "1": {
                "name": "Introduction to Python",
                "time": "60",
                "links": ["www.python.org", "www.w3schools.com/python"]
            },
            "2": {
                "name": "Variables and Data Types",
                "time": "60",
                "links": ["www.w3schools.com/python/python_variables.php"]
            },
            "3": {
                "name": "Control Flow",
                "time": "60",
                "links": ["www.programiz.com/python-programming/if-elif-else"]
            },
            "4": {
                "name": "Functions",
                "time": "60",
                "links": ["www.learnpython.org/en/Functions"]
            },
            "5": {
                "name": "Lists and Tuples",
                "time": "60",
                "links": ["www.w3schools.com/python/python_lists.php"]
            },
            "6": {
                